#Get Weekday Commute Estimates from Google

This is a little script that will take your home and work address, and write a CSV file of estimated commute times from Google's Traffic model accessed from their Distance Matrix API. 

First set up Google Developer account and API keys for Geocode and Distance Matrix, and download the Python wrapper available here: https://github.com/googlemaps/google-maps-services-python.git

In [1]:
#!/usr/bin/env python

import googlemaps
import datetime
import pandas as pd
import numpy as np
import time

### Departure Time in Google Maps API query has to be in the future, so need to get today's date time, and pick the nearest Monday-Friday times that are in the future

In [ ]:
def futuredays(number):
    today = datetime.date.today()
    futureday = today.replace(day = today.day+number)
    return futureday
#example: tomorrow = futuredays(1)

In [ ]:
#Generate date objects for the next 7 days from the current day
next7days = [futuredays(i) for i in range(1,8)]

next7daysofweek = [d.weekday() for d in next7days] 

###Get the date for each weekday 
date.weekday will return day of the week for date as integer: Monday is 0 and Sunday is 6

In [ ]:
mon = next7days[next7daysofweek.index(0)]
tues = next7days[next7daysofweek.index(1)]
wed = next7days[next7daysofweek.index(2)]
thurs = next7days[next7daysofweek.index(3)]
fri = next7days[next7daysofweek.index(4)]

In [ ]:
def getmorningcommutetimes(dayname):
    times = {}
    times['8'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 8)
    times['8:30'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 8, 30)
    times['9'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 9)
    times['9:30'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 9, 30)
    times['10'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 10)
    return times

In [ ]:
def geteveningcommutetimes(dayname):
    times = {}
    times['5'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 17)
    times['5:30'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 17, 30)
    times['6'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 18)
    times['6:30'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 18, 30)
    times['7'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 19)
    times['7:30'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 19,30)
    times['8'] = datetime.datetime(dayname.year, dayname.month, dayname.day, 20)
    return times  

###Create dictionary containing all the datetime objects that we'll use in our queries

In [ ]:
daynames = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']
alltimes = {}
daycount = 0
for wkday in [mon, tues, wed, thurs, fri]:
    morning = getmorningcommutetimes(wkday)
    evening = geteveningcommutetimes(wkday)
    alltimes[daynames[daycount]]=[morning,evening]
    daycount += 1

In [ ]:
gmaps = googlemaps.Client(key = "yourkey")

home = gmaps.geocode('2100 Latham Street, Mountain View CA')
work = gmaps.geocode('3011 North 1st Street, San Jose CA')
#The distance_matrix service will convert strings to lat-long for you but I'm doing it here so we don't have to do it every time in the query (might cost as an extra query towards the usage limit?)

In [ ]:
#Get dict containing lat and lon for that address
home_coord = home[0]['geometry']['location']
work_coord = work[0]['geometry']['location']

## Getting all the morning times

In distance matrix queries the argument order is: origin, destination, so morning times should be (home, work) and evening times (work,home)

In [ ]:
for d in daynames:
    for k in alltimes[d][0].keys():
      optimistic = gmaps.distance_matrix(home_coord, work_coord, mode="driving", units="imperial", departure_time=alltimes[d][0][k], traffic_model="optimistic")         
      pessimistic = gmaps.distance_matrix(home_coord, work_coord, mode="driving", units="imperial", departure_time=alltimes[d][0][k], traffic_model="pessimistic")
      optim_time = optimistic['rows'][0]['elements'][0]['duration_in_traffic']['value']
      pessim_time = pessimistic['rows'][0]['elements'][0]['duration_in_traffic']['value']
      alltimes[d][0][k] = [alltimes[d][0][k], optim_time, pessim_time]

##Wait 10 seconds
The free API is limited to 100 elements per 10 seconds, 2500 elements / day, where # of Elements = # of origins x # of destinations

Finding all the morning commute times = 7 days x 5 times x 2 (optimistic/pessimistic) = 70. There are 7 evening times so = 98. So we have to make the script wait between these two loops.

In [ ]:
time.sleep(10) #Waits 10 seconds

##Getting all the evening times

In [ ]:
for d in daynames:
    for k in alltimes[d][1].keys():
      optimistic = gmaps.distance_matrix(work_coord, home_coord, mode="driving", units="imperial", departure_time=alltimes[d][1][k], traffic_model="optimistic")         
      pessimistic = gmaps.distance_matrix(work_coord, home_coord, mode="driving", units="imperial", departure_time=alltimes[d][1][k], traffic_model="pessimistic")
      optim_time = optimistic['rows'][0]['elements'][0]['duration_in_traffic']['value']
      pessim_time = pessimistic['rows'][0]['elements'][0]['duration_in_traffic']['value']
      alltimes[d][1][k] = [alltimes[d][1][k], optim_time, pessim_time]

##Create DataFrame of Commute times for analysis

In [ ]:
dayofweek  = []
whichcommute = []
timeofday = []
date_time = []
traffic = []
estimate = []
for d in alltimes.keys(): #for each day of the week
    for k in alltimes[d][0].keys():
        dayofweek.extend([d,d])
        whichcommute.extend(["morning","morning"])
        timeofday.extend([k,k])
        date_time.extend([str(alltimes[d][0][k][0]), str(alltimes[d][0][k][0])])
        traffic.append('good')
        estimate.append(float(alltimes[d][0][k][1])/float(60)) # seconds -> minutes
        traffic.append('bad')
        estimate.append(float(alltimes[d][0][k][2])/float(60)) # seconds -> minutes
    for k in alltimes[d][1].keys():
        dayofweek.extend([d,d])
        whichcommute.extend(["evening","evening"])
        timeofday.extend([k,k])
        date_time.extend([str(alltimes[d][1][k][0]),str(alltimes[d][1][k][0])])
        traffic.append('good')
        estimate.append(float(alltimes[d][1][k][1])/float(60)) # seconds -> minutes
        traffic.append('bad')
        estimate.append(float(alltimes[d][1][k][2])/float(60)) # seconds -> minutes

In [ ]:
alltimes_df = pd.DataFrame({'DayofWeek':dayofweek, 'WhichCommute':whichcommute, 'TimeofDay':timeofday, 'Traffic':traffic, 'Estimate':estimate}, index = pd.to_datetime(date_time))

In [ ]:
#Optional write to CSV 
alltimes_df.to_csv("yourfilenamehere")

##Plot stuff

In [ ]:
morningcommute = alltimes_df[[u'Estimate']][alltimes_df['WhichCommute']=="morning"]
drivingtimes = alltimes_df[[u'Estimate']]
drivingtimes['Hour'] = alltimes_df.index.hour
drivingtimes.groupby('Hour').aggregate(np.median).plot()

In [ ]:
import seaborn as sns
sns.regplot(x="Hour", y="Estimate", fit_reg=False, data=drivingtimes)

alltimes_df.groupby('TimeofDay').aggregate(np.median).plot()

In [ ]:
alltimes_df['TimeofDay'] = alltimes_df['TimeofDay'].astype("category", categories=['8', '8:30', '9', '9:30', '10', '5', '5:30', '6','6:30', '7', '7:30'], ordered=False)
sns.stripplot(x="TimeofDay", y="Estimate", data=alltimes_df)